In [1]:
from PIL import Image
import numpy as np
import os

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import Dataset
from torchvision import models

device = torch.device("cuda:0")

In [2]:
class brain_Dataset(Dataset):
    def __init__ (self, data, transform=None):
        self.data = data
        self.transform = transform
        
    def __len__(self):
        return len(self.data[1])
        
    def __getitem__(self, index):
        img = self.data[0][index]
        label = self.data[1][index]
        
        if self.transform:
            img = self.transform(img)
        
        return img, label

In [3]:
def read_images(directory, img_size):
    #logging.info('pytorch_utils.read_images')

    list_img = []
    labels = []

    for name in os.listdir(directory + 'yes'):
        img = Image.open(directory + 'yes/'+ name)
        img = img.resize((img_size, img_size))
        list_img.append(np.asarray(img))
        labels.append(1)
        
    for name in os.listdir(directory + 'no'):
        img = Image.open(directory + 'no/'+ name)
        img = img.resize((img_size, img_size))
        list_img.append(np.asarray(img))
        labels.append(0)

    return list_img, labels

In [4]:
def create_dataloader(directory, img_size, batch_size, transforms = None, validation_split = 0.2):
    #logging.info('pytorch_utils.create_dataloader')

    list_img, labels = read_images(directory, img_size)
    dataset = brain_Dataset([list_img, labels], transforms)

    data_size = len(list_img)
    validation_split = validation_split
    split = int(np.floor(validation_split * data_size))
    indices = list(range(data_size))
    np.random.shuffle(indices)

    train_indices, val_indices = indices[split:], indices[:split]

    train_sampler = SubsetRandomSampler(train_indices)
    val_sampler = SubsetRandomSampler(val_indices)

    train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, 
                                            sampler=train_sampler)
    val_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                            sampler=val_sampler)

    return train_loader, val_loader

In [5]:
def compute_accuracy(model, val_loader):
    model.eval() # Evaluation mode
    
    correct_samples = 0
    total_samples = 0
    
    for i_step, (x, y) in enumerate(val_loader):
      x_gpu = x.to(device, dtype=torch.float)
      y_gpu = y.to(device, dtype=torch.long)

      predictions = model(x_gpu)
      _, indices = torch.max(predictions, 1)

      correct_samples += torch.sum(indices == y_gpu)
      total_samples += y.shape[0]
      
    accuracy = float(correct_samples)/total_samples       
    
    return accuracy

In [6]:
def train_model(model, train_loader, val_loader, loss, optimizer, num_epochs):    
    loss_history = []
    train_history = []
    val_history = []
    
    for epoch in range(num_epochs):
        model.train() # Enter train mode
        
        loss_accum = 0
        correct_samples = 0
        total_samples = 0
        for i_step, (x, y) in enumerate(train_loader):
            x_gpu = x.to(device, dtype=torch.float)
            y_gpu = y.to(device, dtype=torch.long)
            prediction = model(x_gpu)   
            loss_value = loss(prediction, y_gpu)
            optimizer.zero_grad()
            loss_value.backward()
            optimizer.step()
            
            _, indices = torch.max(prediction, 1)

            correct_samples += torch.sum(indices == y_gpu)
            total_samples += y.shape[0]
            
            loss_accum += loss_value

        ave_loss = loss_accum / i_step
        train_accuracy = float(correct_samples) / total_samples
        val_accuracy = compute_accuracy(model, val_loader)
        
        loss_history.append(float(ave_loss))
        train_history.append(train_accuracy)
        val_history.append(val_accuracy)
        
        print("Average loss: %f, Train accuracy: %f, Val accuracy: %f" % (ave_loss, train_accuracy, val_accuracy))
        
    #return loss_history, train_history, val_history

In [7]:
transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])                           
                       ])
directory = 'new_data_norm/'
train_loader, val_loader = create_dataloader(directory = directory, img_size = 240, batch_size = 8, transforms = transform)

# Alexnet

In [44]:
model = models.alexnet(pretrained=True)

model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, 2)

model = model.to(device)

## SGD

In [14]:
loss = nn.CrossEntropyLoss().type(torch.cuda.FloatTensor)
optimizer = optim.SGD(model.parameters(), lr=1e-3)

In [15]:
train_model(model, train_loader, val_loader, loss, optimizer, 20)

Average loss: 0.514698, Train accuracy: 0.750958, Val accuracy: 0.749361
Average loss: 0.357534, Train accuracy: 0.840358, Val accuracy: 0.872123
Average loss: 0.267262, Train accuracy: 0.880587, Val accuracy: 0.895141
Average loss: 0.192739, Train accuracy: 0.928480, Val accuracy: 0.933504
Average loss: 0.166694, Train accuracy: 0.939336, Val accuracy: 0.933504
Average loss: 0.142137, Train accuracy: 0.948914, Val accuracy: 0.915601
Average loss: 0.110696, Train accuracy: 0.961686, Val accuracy: 0.948849
Average loss: 0.086801, Train accuracy: 0.971264, Val accuracy: 0.961637
Average loss: 0.084631, Train accuracy: 0.971264, Val accuracy: 0.928389
Average loss: 0.054509, Train accuracy: 0.985313, Val accuracy: 0.969309
Average loss: 0.057994, Train accuracy: 0.983397, Val accuracy: 0.951407
Average loss: 0.041683, Train accuracy: 0.985951, Val accuracy: 0.969309
Average loss: 0.047603, Train accuracy: 0.988506, Val accuracy: 0.969309
Average loss: 0.027007, Train accuracy: 0.993614, V

([0.5146983861923218,
  0.35753387212753296,
  0.26726195216178894,
  0.19273947179317474,
  0.1666942983865738,
  0.14213679730892181,
  0.11069601029157639,
  0.08680105209350586,
  0.08463147282600403,
  0.054508719593286514,
  0.05799388512969017,
  0.041682831943035126,
  0.04760296270251274,
  0.02700701169669628,
  0.022317318245768547,
  0.02879493683576584,
  0.018886391073465347,
  0.019816339015960693,
  0.016585152596235275,
  0.017848078161478043],
 [0.7509578544061303,
  0.8403575989782887,
  0.8805874840357599,
  0.9284802043422733,
  0.9393358876117497,
  0.9489144316730523,
  0.9616858237547893,
  0.9712643678160919,
  0.9712643678160919,
  0.9853128991060025,
  0.9833971902937421,
  0.9859514687100894,
  0.9885057471264368,
  0.9936143039591315,
  0.9955300127713921,
  0.9897828863346104,
  0.9948914431673053,
  0.9942528735632183,
  0.9961685823754789,
  0.9948914431673053],
 [0.7493606138107417,
  0.8721227621483376,
  0.8951406649616368,
  0.9335038363171355,
  0.9

In [25]:
train_model(model, train_loader, val_loader, loss, optimizer, 30)

Average loss: 0.493495, Train accuracy: 0.747765, Val accuracy: 0.828645
Average loss: 0.371374, Train accuracy: 0.839080, Val accuracy: 0.872123
Average loss: 0.290130, Train accuracy: 0.885696, Val accuracy: 0.721228
Average loss: 0.246343, Train accuracy: 0.901660, Val accuracy: 0.890026
Average loss: 0.191287, Train accuracy: 0.930396, Val accuracy: 0.900256
Average loss: 0.158633, Train accuracy: 0.937420, Val accuracy: 0.951407
Average loss: 0.133471, Train accuracy: 0.955939, Val accuracy: 0.961637
Average loss: 0.104159, Train accuracy: 0.964240, Val accuracy: 0.956522
Average loss: 0.092274, Train accuracy: 0.968710, Val accuracy: 0.974425
Average loss: 0.072014, Train accuracy: 0.977011, Val accuracy: 0.953964
Average loss: 0.063080, Train accuracy: 0.981481, Val accuracy: 0.948849
Average loss: 0.050753, Train accuracy: 0.984674, Val accuracy: 0.971867
Average loss: 0.030440, Train accuracy: 0.992337, Val accuracy: 0.956522
Average loss: 0.041695, Train accuracy: 0.985313, V

([0.4934949576854706,
  0.3713735342025757,
  0.2901299297809601,
  0.24634340405464172,
  0.19128741323947906,
  0.15863323211669922,
  0.13347116112709045,
  0.10415872931480408,
  0.09227357059717178,
  0.07201351970434189,
  0.06308049708604813,
  0.05075305327773094,
  0.030440306290984154,
  0.04169544577598572,
  0.033413492143154144,
  0.02213483676314354,
  0.020914016291499138,
  0.01878846436738968,
  0.025408314540982246,
  0.016497278586030006,
  0.013011381961405277,
  0.018635137006640434,
  0.012413202784955502,
  0.016247356310486794,
  0.012407331727445126,
  0.007424139883369207,
  0.010375847108662128,
  0.010034586302936077,
  0.010201823897659779,
  0.004323864821344614],
 [0.7477650063856961,
  0.8390804597701149,
  0.8856960408684547,
  0.9016602809706258,
  0.9303959131545338,
  0.9374201787994891,
  0.9559386973180076,
  0.9642401021711366,
  0.9687100893997446,
  0.9770114942528736,
  0.9814814814814815,
  0.9846743295019157,
  0.9923371647509579,
  0.9853128

## Adam

In [9]:
loss = nn.CrossEntropyLoss().type(torch.cuda.FloatTensor)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [10]:
train_model(model, train_loader, val_loader, loss, optimizer, 10)

Average loss: 0.755396, Train accuracy: 0.506386, Val accuracy: 0.531969
Average loss: 0.701701, Train accuracy: 0.498084, Val accuracy: 0.531969
Average loss: 0.700313, Train accuracy: 0.502554, Val accuracy: 0.531969
Average loss: 0.701680, Train accuracy: 0.496807, Val accuracy: 0.531969
Average loss: 0.701574, Train accuracy: 0.491699, Val accuracy: 0.468031
Average loss: 0.701008, Train accuracy: 0.490421, Val accuracy: 0.468031
Average loss: 0.700492, Train accuracy: 0.492976, Val accuracy: 0.468031
Average loss: 0.700116, Train accuracy: 0.505109, Val accuracy: 0.531969
Average loss: 0.700989, Train accuracy: 0.491060, Val accuracy: 0.468031
Average loss: 0.700986, Train accuracy: 0.498723, Val accuracy: 0.531969


([0.7553964853286743,
  0.7017010450363159,
  0.7003127336502075,
  0.7016804814338684,
  0.7015736103057861,
  0.7010084390640259,
  0.7004918456077576,
  0.7001155018806458,
  0.7009887099266052,
  0.7009863257408142],
 [0.5063856960408685,
  0.49808429118773945,
  0.5025542784163474,
  0.49680715197956576,
  0.49169859514687103,
  0.4904214559386973,
  0.4929757343550447,
  0.5051085568326947,
  0.49106002554278416,
  0.4987228607918263],
 [0.5319693094629157,
  0.5319693094629157,
  0.5319693094629157,
  0.5319693094629157,
  0.4680306905370844,
  0.4680306905370844,
  0.4680306905370844,
  0.5319693094629157,
  0.4680306905370844,
  0.5319693094629157])

## Adagrad 

In [21]:
loss = nn.CrossEntropyLoss().type(torch.cuda.FloatTensor)
optimizer = optim.Adagrad(model.parameters(), lr=1e-3)

In [22]:
train_model(model, train_loader, val_loader, loss, optimizer, 15)

Average loss: 0.655460, Train accuracy: 0.656450, Val accuracy: 0.567775
Average loss: 0.588020, Train accuracy: 0.685185, Val accuracy: 0.672634
Average loss: 0.538579, Train accuracy: 0.724138, Val accuracy: 0.718670
Average loss: 0.498188, Train accuracy: 0.767561, Val accuracy: 0.749361
Average loss: 0.367694, Train accuracy: 0.834610, Val accuracy: 0.841432
Average loss: 0.306939, Train accuracy: 0.886335, Val accuracy: 0.895141
Average loss: 0.209476, Train accuracy: 0.921456, Val accuracy: 0.905371
Average loss: 0.155096, Train accuracy: 0.940613, Val accuracy: 0.928389
Average loss: 0.111104, Train accuracy: 0.955300, Val accuracy: 0.928389
Average loss: 0.107148, Train accuracy: 0.962963, Val accuracy: 0.948849
Average loss: 0.056685, Train accuracy: 0.978289, Val accuracy: 0.948849
Average loss: 0.044035, Train accuracy: 0.989144, Val accuracy: 0.964194
Average loss: 0.031917, Train accuracy: 0.989783, Val accuracy: 0.964194
Average loss: 0.040821, Train accuracy: 0.985951, V

## ASGD

In [24]:
loss = nn.CrossEntropyLoss().type(torch.cuda.FloatTensor)
optimizer = optim.ASGD(model.parameters(), lr=1e-3)

In [25]:
train_model(model, train_loader, val_loader, loss, optimizer, 15)

Average loss: 0.518355, Train accuracy: 0.742018, Val accuracy: 0.838875
Average loss: 0.362620, Train accuracy: 0.830779, Val accuracy: 0.874680
Average loss: 0.280479, Train accuracy: 0.880587, Val accuracy: 0.900256
Average loss: 0.209895, Train accuracy: 0.915709, Val accuracy: 0.843990
Average loss: 0.174861, Train accuracy: 0.933589, Val accuracy: 0.928389
Average loss: 0.151470, Train accuracy: 0.945722, Val accuracy: 0.925831
Average loss: 0.117123, Train accuracy: 0.961047, Val accuracy: 0.672634
Average loss: 0.096002, Train accuracy: 0.968710, Val accuracy: 0.948849
Average loss: 0.073285, Train accuracy: 0.979566, Val accuracy: 0.948849
Average loss: 0.062788, Train accuracy: 0.976373, Val accuracy: 0.941176
Average loss: 0.049828, Train accuracy: 0.983397, Val accuracy: 0.948849
Average loss: 0.046098, Train accuracy: 0.984036, Val accuracy: 0.951407
Average loss: 0.035945, Train accuracy: 0.986590, Val accuracy: 0.930946
Average loss: 0.025608, Train accuracy: 0.994253, V

Pour l'instant parmi les optimiseurs testés SGD et Adagrad sont les mieux pour Alexnet

In [36]:
loss = nn.CrossEntropyLoss().type(torch.cuda.FloatTensor)
optimizer = optim.SGD(model.parameters(), lr=1e-2, momentum = 0.1)

In [37]:
train_model(model, train_loader, val_loader, loss, optimizer, 10)

Average loss: 0.617628, Train accuracy: 0.717752, Val accuracy: 0.843990
Average loss: 0.286149, Train accuracy: 0.875479, Val accuracy: 0.890026
Average loss: 0.155899, Train accuracy: 0.943806, Val accuracy: 0.895141
Average loss: 0.131997, Train accuracy: 0.952746, Val accuracy: 0.966752
Average loss: 0.040783, Train accuracy: 0.986590, Val accuracy: 0.974425
Average loss: 0.140254, Train accuracy: 0.954662, Val accuracy: 0.966752
Average loss: 0.050960, Train accuracy: 0.980204, Val accuracy: 0.969309
Average loss: 0.018420, Train accuracy: 0.995530, Val accuracy: 0.976982
Average loss: 0.005103, Train accuracy: 0.998084, Val accuracy: 0.976982
Average loss: 0.001064, Train accuracy: 1.000000, Val accuracy: 0.982097


In [45]:
loss = nn.CrossEntropyLoss().type(torch.cuda.FloatTensor)
optimizer = optim.SGD(model.parameters(), lr=1e-2, momentum = 0.1, weight_decay = 1e-3)

In [46]:
train_model(model, train_loader, val_loader, loss, optimizer, 10)

Average loss: 0.583586, Train accuracy: 0.742018, Val accuracy: 0.831202
Average loss: 0.280331, Train accuracy: 0.880587, Val accuracy: 0.897698
Average loss: 0.159111, Train accuracy: 0.932312, Val accuracy: 0.953964
Average loss: 0.077307, Train accuracy: 0.971264, Val accuracy: 0.861893
Average loss: 0.050352, Train accuracy: 0.984674, Val accuracy: 0.969309
Average loss: 0.062373, Train accuracy: 0.986590, Val accuracy: 0.961637
Average loss: 0.032089, Train accuracy: 0.988506, Val accuracy: 0.964194
Average loss: 0.022068, Train accuracy: 0.991699, Val accuracy: 0.979540
Average loss: 0.001661, Train accuracy: 1.000000, Val accuracy: 0.982097
Average loss: 0.001000, Train accuracy: 1.000000, Val accuracy: 0.982097


Conclusion paramètres

Utilisation d'Alexnet : Optimiseur : SGD (avec lr = 1e-2, momentum = 0,1, weight_decay = 1e-3) et 10 epochs

Résultats autour de 98%

# Resnet

In [62]:
model = models.resnet18(pretrained=True)

num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)

model = model.to(device)

## SGD

In [51]:
loss = nn.CrossEntropyLoss().type(torch.cuda.FloatTensor)
optimizer = optim.SGD(model.parameters(), lr=1e-3)

In [52]:
train_model(model, train_loader, val_loader, loss, optimizer, 15)

Average loss: 0.591976, Train accuracy: 0.689017, Val accuracy: 0.744246
Average loss: 0.440150, Train accuracy: 0.830140, Val accuracy: 0.867008
Average loss: 0.332178, Train accuracy: 0.884419, Val accuracy: 0.900256
Average loss: 0.266021, Train accuracy: 0.906769, Val accuracy: 0.915601
Average loss: 0.213242, Train accuracy: 0.939336, Val accuracy: 0.936061
Average loss: 0.180145, Train accuracy: 0.946999, Val accuracy: 0.941176
Average loss: 0.151401, Train accuracy: 0.960409, Val accuracy: 0.959079
Average loss: 0.127047, Train accuracy: 0.963602, Val accuracy: 0.961637
Average loss: 0.104890, Train accuracy: 0.975734, Val accuracy: 0.959079
Average loss: 0.100275, Train accuracy: 0.971903, Val accuracy: 0.961637
Average loss: 0.079515, Train accuracy: 0.986590, Val accuracy: 0.964194
Average loss: 0.059939, Train accuracy: 0.990421, Val accuracy: 0.964194
Average loss: 0.054447, Train accuracy: 0.989783, Val accuracy: 0.969309
Average loss: 0.054020, Train accuracy: 0.989783, V

## Adam

In [54]:
loss = nn.CrossEntropyLoss().type(torch.cuda.FloatTensor)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [55]:
train_model(model, train_loader, val_loader, loss, optimizer, 15)

Average loss: 0.516295, Train accuracy: 0.782248, Val accuracy: 0.861893
Average loss: 0.267459, Train accuracy: 0.897190, Val accuracy: 0.920716
Average loss: 0.196430, Train accuracy: 0.930396, Val accuracy: 0.792839
Average loss: 0.186117, Train accuracy: 0.933589, Val accuracy: 0.867008
Average loss: 0.106293, Train accuracy: 0.961686, Val accuracy: 0.618926
Average loss: 0.123815, Train accuracy: 0.961047, Val accuracy: 0.943734
Average loss: 0.115284, Train accuracy: 0.957216, Val accuracy: 0.861893
Average loss: 0.131293, Train accuracy: 0.956577, Val accuracy: 0.923274
Average loss: 0.053452, Train accuracy: 0.980843, Val accuracy: 0.943734
Average loss: 0.030763, Train accuracy: 0.989783, Val accuracy: 0.951407
Average loss: 0.129103, Train accuracy: 0.948914, Val accuracy: 0.946292
Average loss: 0.065848, Train accuracy: 0.977650, Val accuracy: 0.943734
Average loss: 0.034322, Train accuracy: 0.987867, Val accuracy: 0.969309
Average loss: 0.024529, Train accuracy: 0.992976, V

## Adagrad 

In [57]:
loss = nn.CrossEntropyLoss().type(torch.cuda.FloatTensor)
optimizer = optim.Adagrad(model.parameters(), lr=1e-3)

In [58]:
train_model(model, train_loader, val_loader, loss, optimizer, 15)

Average loss: 0.273051, Train accuracy: 0.890805, Val accuracy: 0.966752
Average loss: 0.058163, Train accuracy: 0.982120, Val accuracy: 0.953964
Average loss: 0.029089, Train accuracy: 0.991060, Val accuracy: 0.989770
Average loss: 0.006073, Train accuracy: 1.000000, Val accuracy: 0.987212
Average loss: 0.011127, Train accuracy: 0.998723, Val accuracy: 0.997442
Average loss: 0.012957, Train accuracy: 0.997446, Val accuracy: 0.987212
Average loss: 0.007869, Train accuracy: 0.997446, Val accuracy: 0.997442
Average loss: 0.003448, Train accuracy: 1.000000, Val accuracy: 0.997442
Average loss: 0.002842, Train accuracy: 0.999361, Val accuracy: 0.994885
Average loss: 0.001648, Train accuracy: 1.000000, Val accuracy: 0.997442
Average loss: 0.001718, Train accuracy: 1.000000, Val accuracy: 0.992327
Average loss: 0.003281, Train accuracy: 0.999361, Val accuracy: 0.989770
Average loss: 0.001784, Train accuracy: 1.000000, Val accuracy: 0.994885
Average loss: 0.001596, Train accuracy: 1.000000, V

In [63]:
loss = nn.CrossEntropyLoss().type(torch.cuda.FloatTensor)
optimizer = optim.Adagrad(model.parameters(), lr=1e-3, weight_decay = 1e-3)

In [64]:
train_model(model, train_loader, val_loader, loss, optimizer, 6)

Average loss: 0.278530, Train accuracy: 0.895275, Val accuracy: 0.815857
Average loss: 0.049927, Train accuracy: 0.984674, Val accuracy: 0.984655
Average loss: 0.014998, Train accuracy: 0.997446, Val accuracy: 0.994885
Average loss: 0.014087, Train accuracy: 0.995530, Val accuracy: 0.984655
Average loss: 0.007627, Train accuracy: 0.998723, Val accuracy: 0.989770
Average loss: 0.011993, Train accuracy: 0.997446, Val accuracy: 0.994885


Conclusion paramètres

Utilisation de Resnet : Optimiseur : Adagrad (avec lr = 1e-3, weight_decay = 1e-3) et 6 epochs

Résultats autour de 99%

# VGG (attention très long à train comparé aux autres)

In [14]:
model = models.vgg16(pretrained=True)

num_features = model.classifier[6].in_features
features = list(model.classifier.children())[:-1] # Remove last layer
features.extend([nn.Linear(num_features, 2)]) # Add our layer with 4 outputs
model.classifier = nn.Sequential(*features) # Replace the model classifie

model = model.to(device)

## SGD

In [11]:
loss = nn.CrossEntropyLoss().type(torch.cuda.FloatTensor)
optimizer = optim.SGD(model.parameters(), lr=1e-3)

In [12]:
train_model(model, train_loader, val_loader, loss, optimizer, 15)

Average loss: 0.565571, Train accuracy: 0.698595, Val accuracy: 0.762148
Average loss: 0.353250, Train accuracy: 0.854406, Val accuracy: 0.874680
Average loss: 0.218821, Train accuracy: 0.922095, Val accuracy: 0.895141
Average loss: 0.151999, Train accuracy: 0.941252, Val accuracy: 0.895141
Average loss: 0.094103, Train accuracy: 0.971264, Val accuracy: 0.961637
Average loss: 0.074226, Train accuracy: 0.976373, Val accuracy: 0.979540
Average loss: 0.043693, Train accuracy: 0.989144, Val accuracy: 0.982097
Average loss: 0.031927, Train accuracy: 0.993614, Val accuracy: 0.974425
Average loss: 0.017049, Train accuracy: 0.994891, Val accuracy: 0.987212
Average loss: 0.010985, Train accuracy: 0.997446, Val accuracy: 0.984655
Average loss: 0.036416, Train accuracy: 0.989783, Val accuracy: 0.982097
Average loss: 0.008564, Train accuracy: 0.999361, Val accuracy: 0.982097
Average loss: 0.006503, Train accuracy: 0.998723, Val accuracy: 0.989770
Average loss: 0.004952, Train accuracy: 0.999361, V

## Adam

In [9]:
loss = nn.CrossEntropyLoss().type(torch.cuda.FloatTensor)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [10]:
train_model(model, train_loader, val_loader, loss, optimizer, 15)

Average loss: 0.805061, Train accuracy: 0.496169, Val accuracy: 0.496164
Average loss: 0.711043, Train accuracy: 0.508940, Val accuracy: 0.496164
Average loss: 0.709715, Train accuracy: 0.518519, Val accuracy: 0.503836
Average loss: 0.704796, Train accuracy: 0.499361, Val accuracy: 0.496164
Average loss: 0.705761, Train accuracy: 0.479566, Val accuracy: 0.496164
Average loss: 0.705163, Train accuracy: 0.522989, Val accuracy: 0.503836
Average loss: 0.703874, Train accuracy: 0.493614, Val accuracy: 0.496164
Average loss: 0.701343, Train accuracy: 0.515326, Val accuracy: 0.496164
Average loss: 0.702680, Train accuracy: 0.475096, Val accuracy: 0.496164
Average loss: 0.703702, Train accuracy: 0.491060, Val accuracy: 0.496164
Average loss: 0.700990, Train accuracy: 0.492337, Val accuracy: 0.496164
Average loss: 0.702095, Train accuracy: 0.494891, Val accuracy: 0.503836
Average loss: 0.700202, Train accuracy: 0.501916, Val accuracy: 0.503836
Average loss: 0.700307, Train accuracy: 0.501277, V

## Adagrad 

In [9]:
loss = nn.CrossEntropyLoss().type(torch.cuda.FloatTensor)
optimizer = optim.Adagrad(model.parameters(), lr=1e-3)

In [10]:
train_model(model, train_loader, val_loader, loss, optimizer, 15)

Average loss: 0.711855, Train accuracy: 0.585568, Val accuracy: 0.603581
Average loss: 0.507839, Train accuracy: 0.763729, Val accuracy: 0.820972
Average loss: 0.245199, Train accuracy: 0.900383, Val accuracy: 0.795396
Average loss: 0.097116, Train accuracy: 0.964240, Val accuracy: 0.872123
Average loss: 0.022857, Train accuracy: 0.994253, Val accuracy: 0.961637
Average loss: 0.003239, Train accuracy: 0.998723, Val accuracy: 0.971867
Average loss: 0.011875, Train accuracy: 0.996169, Val accuracy: 0.971867
Average loss: 0.000241, Train accuracy: 1.000000, Val accuracy: 0.974425
Average loss: 0.000040, Train accuracy: 1.000000, Val accuracy: 0.974425
Average loss: 0.000018, Train accuracy: 1.000000, Val accuracy: 0.976982
Average loss: 0.000005, Train accuracy: 1.000000, Val accuracy: 0.974425
Average loss: 0.000012, Train accuracy: 1.000000, Val accuracy: 0.971867
Average loss: 0.000004, Train accuracy: 1.000000, Val accuracy: 0.974425
Average loss: 0.000003, Train accuracy: 1.000000, V

In [15]:
loss = nn.CrossEntropyLoss().type(torch.cuda.FloatTensor)
optimizer = optim.SGD(model.parameters(), lr=1e-2)

In [16]:
train_model(model, train_loader, val_loader, loss, optimizer, 15)

Average loss: 0.579351, Train accuracy: 0.701149, Val accuracy: 0.887468
Average loss: 0.199586, Train accuracy: 0.924649, Val accuracy: 0.913043
Average loss: 0.099438, Train accuracy: 0.969987, Val accuracy: 0.976982
Average loss: 0.018597, Train accuracy: 0.994891, Val accuracy: 0.987212
Average loss: 0.001250, Train accuracy: 1.000000, Val accuracy: 0.987212
Average loss: 0.000216, Train accuracy: 1.000000, Val accuracy: 0.987212
Average loss: 0.000222, Train accuracy: 1.000000, Val accuracy: 0.987212
Average loss: 0.000096, Train accuracy: 1.000000, Val accuracy: 0.989770
Average loss: 0.000088, Train accuracy: 1.000000, Val accuracy: 0.989770
Average loss: 0.000067, Train accuracy: 1.000000, Val accuracy: 0.989770
Average loss: 0.000037, Train accuracy: 1.000000, Val accuracy: 0.989770
Average loss: 0.000043, Train accuracy: 1.000000, Val accuracy: 0.989770
Average loss: 0.000037, Train accuracy: 1.000000, Val accuracy: 0.989770
Average loss: 0.000031, Train accuracy: 1.000000, V

Conclusion paramètres

Utilisation de VGG : Optimiseur : SGD (avec lr = 1e-2) et 5 epochs

Résultats autour de 98%


## Fixed feature extractor

In [47]:
model = models.alexnet(pretrained=True)

for param in model.parameters():
    param.requires_grad = False
    

model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, 2)

model = model.to(device)

In [48]:
loss = nn.CrossEntropyLoss().type(torch.cuda.FloatTensor)
optimizer = optim.SGD(model.parameters(), lr=1e-3)

In [49]:
train_model(model, train_loader, val_loader, loss, optimizer, 25)

Average loss: 0.560077, Train accuracy: 0.724138, Val accuracy: 0.593350
Average loss: 0.456238, Train accuracy: 0.786079, Val accuracy: 0.810742
Average loss: 0.419200, Train accuracy: 0.818646, Val accuracy: 0.820972
Average loss: 0.392038, Train accuracy: 0.829502, Val accuracy: 0.851662
Average loss: 0.369810, Train accuracy: 0.828225, Val accuracy: 0.838875
Average loss: 0.328132, Train accuracy: 0.856960, Val accuracy: 0.820972
Average loss: 0.350224, Train accuracy: 0.846105, Val accuracy: 0.769821
Average loss: 0.335526, Train accuracy: 0.860792, Val accuracy: 0.861893
Average loss: 0.320879, Train accuracy: 0.864623, Val accuracy: 0.884910
Average loss: 0.314250, Train accuracy: 0.863985, Val accuracy: 0.884910
Average loss: 0.312332, Train accuracy: 0.861430, Val accuracy: 0.877238
Average loss: 0.310747, Train accuracy: 0.868455, Val accuracy: 0.890026
Average loss: 0.293823, Train accuracy: 0.878033, Val accuracy: 0.856777
Average loss: 0.303417, Train accuracy: 0.871009, V

Résultats bcp moins bien en fixed feature extractor, autant faire du fine tuning même si moins rapide